In [43]:
import os
from tokenizers import Tokenizer, models, pre_tokenizers, decoders, trainers, processors
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.normalizers import NFD, StripAccents
import pandas as pd

import torch

In [39]:
caption = []
with open('dataset/crohme/train/caption.txt', 'r') as f:
    data = f.readlines()
    for line in data:
        line = line.strip()
        if '\t' in line:
            _, label = line.split('\t', 1)  # Split by the first tab
        else:
            # Skip if improperly formatted
            continue
        caption.append(label)
caption[400:405], caption[-5:]

(['\\beta', 'h B', 'p', 'x l', 'm'],
 ['x = a _ { 0 } + \\frac { 1 } { a _ { 1 } + \\frac { 1 } { a _ { 2 } + \\frac { 1 } { a _ { 3 } + \\ldots } } }',
  '\\frac { \\sin x } { x } = \\frac { \\sin x - \\sin 0 } { x - 0 }',
  '\\alpha ^ { n } + a _ { n - 1 } \\alpha ^ { n - 1 } + \\ldots + a _ { 1 } \\alpha + a _ { 0 } = 0',
  '\\frac { n _ { 1 } } { a _ { 1 } } - \\frac { n _ { 2 } } { a _ { 2 } } = \\frac { n _ { 1 } - n _ { 2 } } { c }',
  'x _ { 1 } ^ { 2 } + \\ldots + x _ { i } ^ { 2 } - x _ { i + 1 } ^ { 2 } - \\ldots - x _ { j } ^ { 2 } = x _ { n }'])

In [3]:
dictionary = []
with open('dataset/crohme/crohme_dictionary.txt', 'r') as f:
    data = f.readlines()
    dictionary.extend([line.strip() for line in data])

dictionary

['!',
 '(',
 ')',
 '+',
 ',',
 '-',
 '.',
 '/',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '<',
 '=',
 '>',
 'A',
 'B',
 'C',
 'E',
 'F',
 'G',
 'H',
 'I',
 'L',
 'M',
 'N',
 'P',
 'R',
 'S',
 'T',
 'V',
 'X',
 'Y',
 '[',
 '\\Delta',
 '\\Pi',
 '\\alpha',
 '\\beta',
 '\\cdot',
 '\\cdots',
 '\\cos',
 '\\div',
 '\\exists',
 '\\forall',
 '\\frac',
 '\\gamma',
 '\\geq',
 '\\in',
 '\\infty',
 '\\int',
 '\\lambda',
 '\\ldots',
 '\\leq',
 '\\lim',
 '\\limits',
 '\\log',
 '\\mu',
 '\\neq',
 '\\phi',
 '\\pi',
 '\\pm',
 '\\prime',
 '\\rightarrow',
 '\\sigma',
 '\\sin',
 '\\sqrt',
 '\\sum',
 '\\tan',
 '\\theta',
 '\\times',
 '\\{',
 '\\}',
 ']',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '|',
 '{',
 '}',
 '^',
 '_']

In [33]:
tokenizer = Tokenizer(BPE())

trainer = BpeTrainer(vocab=dictionary, speical_tokens=['[PAD]', '[UNK]', '[CLS]', '[SEP]', '[MASK]'])
tokenizer.pre_tokenizer = pre_tokenizers.Sequence([Whitespace()])
tokenizer.train_from_iterator(caption, trainer=trainer)

tokenizer.save('dataset/crohme/train/custom_tokenizer.json')

Ignored unknown kwargs option vocab
Ignored unknown kwargs option speical_tokens





In [19]:
# type(tokenizer.get_vocab())
dict(sorted(tokenizer.get_vocab().items(), key=lambda item: item[1]))

{'!': 0,
 '(': 1,
 ')': 2,
 '+': 3,
 ',': 4,
 '-': 5,
 '.': 6,
 '/': 7,
 '0': 8,
 '1': 9,
 '2': 10,
 '3': 11,
 '4': 12,
 '5': 13,
 '6': 14,
 '7': 15,
 '8': 16,
 '9': 17,
 '<': 18,
 '=': 19,
 '>': 20,
 'A': 21,
 'B': 22,
 'C': 23,
 'D': 24,
 'E': 25,
 'F': 26,
 'G': 27,
 'H': 28,
 'I': 29,
 'L': 30,
 'M': 31,
 'N': 32,
 'P': 33,
 'R': 34,
 'S': 35,
 'T': 36,
 'V': 37,
 'X': 38,
 'Y': 39,
 '[': 40,
 '\\': 41,
 ']': 42,
 '^': 43,
 '_': 44,
 'a': 45,
 'b': 46,
 'c': 47,
 'd': 48,
 'e': 49,
 'f': 50,
 'g': 51,
 'h': 52,
 'i': 53,
 'j': 54,
 'k': 55,
 'l': 56,
 'm': 57,
 'n': 58,
 'o': 59,
 'p': 60,
 'q': 61,
 'r': 62,
 's': 63,
 't': 64,
 'u': 65,
 'v': 66,
 'w': 67,
 'x': 68,
 'y': 69,
 'z': 70,
 '{': 71,
 '|': 72,
 '}': 73,
 'ra': 74,
 'fra': 75,
 'frac': 76,
 'im': 77,
 'qr': 78,
 'sqr': 79,
 'sqrt': 80,
 'in': 81,
 'ta': 82,
 'lim': 83,
 'ts': 84,
 'its': 85,
 'limits': 86,
 'eta': 87,
 'sin': 88,
 'ime': 89,
 'su': 90,
 'sum': 91,
 'int': 92,
 'time': 93,
 'times': 94,
 'co': 95,
 'cos

In [32]:
print(f"Vocabulary size: {tokenizer.get_vocab_size()}")
# print(f"Token for '{chr(92)}frac': {tokenizer.token_to_id(f'{chr(92)}frac')}")
tokenizer.decode(tokenizer.encode('\\sqrt').ids)  # '\\ sqrt'

Vocabulary size: 159


'\\ sqrt'

In [44]:
torch.tensor(tokenizer.encode('\cos x + i \sin x = e ^ { i x }').ids, dtype=torch.long)

tensor([41, 96, 68,  3, 53, 41, 88, 68, 19, 49, 43, 71, 53, 68, 73])

In [42]:
tokenizer.encode('\\frac { \\sin x } { x } = \\frac { \\sin x - \\sin 0 } { x - 0 }').ids

[41,
 76,
 71,
 41,
 88,
 68,
 73,
 71,
 68,
 73,
 19,
 41,
 76,
 71,
 41,
 88,
 68,
 5,
 41,
 88,
 8,
 73,
 71,
 68,
 5,
 8,
 73]